In [6]:
import numpy as np
import cv2
import os
from xml.dom import minidom

In [7]:
def readImage(imagePath):
    # Loads image:
    inputImage = cv2.imread(imagePath)
    # Checks if image was successfully loaded:
    if inputImage is None:
        print("readImage>> Error: Could not load Input image.")
    return inputImage


# Defines a re-sizable image window:
def showImage(imageName, inputImage):
    cv2.namedWindow(imageName, cv2.WINDOW_NORMAL)
    cv2.imshow(imageName, inputImage)
    cv2.waitKey(0)
    cv2.destroyWindow(imageName)


# Writes an PNG image:
def writeImage(imagePath, inputImage):
    imagePath = imagePath + ".png"
    cv2.imwrite(imagePath, inputImage, [cv2.IMWRITE_PNG_COMPRESSION, 0])
    print("Wrote Image: " + imagePath)


In [8]:
imgPath = os.path.join(os.getcwd(), 'Images')
binImgPath = os.path.join(imgPath, 'Binary')
colorImgPath = os.path.join(imgPath, 'Color')
print("The files insde ", binImgPath, "are: ")
#inputImage = readImage(picPath)
#showImage("fire10",inputImage)
root = minidom.Document()
xml = root.createElement('root') 

The files insde  d:\proyectorobotica\RoboticsProject\Images\Binary are: 


In [9]:
colorList = os.listdir(colorImgPath)
#for elem in lista:
    #print(elem)
binList = os.listdir(binImgPath)
#for elem in lista:
#    print(elem)

In [10]:
#Check first image and check the bounding rect of the blob
colorInputImage = readImage(os.path.join(colorImgPath, colorList[0]))
showImage(colorList[0], colorInputImage)
binInputImage = readImage(os.path.join(binImgPath, binList[0]))
binInputImage = cv2.cvtColor(binInputImage, cv2.COLOR_BGR2GRAY)
automaticThreshold, binInputImage = cv2.threshold(binInputImage, 0, 255, cv2.THRESH_OTSU)
showImage(binList[0], binInputImage)
print(binInputImage)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [13]:
contours, _ = cv2.findContours(binInputImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(binInputImage.shape)
for c in contours:
    # Compute blob bounding rectangle:
    blobRectangle = cv2.boundingRect(c)
    # Compute blob area:
    blobArea = cv2.contourArea(c)
    # Print the blob area:
    print("Blob Area: "+str(blobArea))
    # Print blobRectangle
    print("BlobRectangle List: ", blobRectangle)
    cv2.rectangle(colorInputImage, (blobRectangle[0], blobRectangle[1]), (blobRectangle[0]+blobRectangle[2], blobRectangle[1]+blobRectangle[3]), (0, 128, 0), 2)
showImage("Color", colorInputImage)


(450, 800)
Blob Area: 40.5
BlobRectangle List:  (339, 344, 12, 8)
Blob Area: 31.0
BlobRectangle List:  (355, 333, 5, 11)
Blob Area: 7.0
BlobRectangle List:  (388, 248, 5, 4)
Blob Area: 16.5
BlobRectangle List:  (380, 239, 6, 7)
Blob Area: 50.5
BlobRectangle List:  (394, 238, 13, 10)
Blob Area: 4.5
BlobRectangle List:  (390, 234, 4, 3)
Blob Area: 449.5
BlobRectangle List:  (263, 228, 23, 48)
Blob Area: 5971.5
BlobRectangle List:  (289, 205, 95, 118)


In [14]:
from pascal_voc_writer import Writer

# create pascal voc writer (image_path, width, height)
writer = Writer(os.path.join(colorImgPath, colorList[0]), colorInputImage.shape[1], colorInputImage.shape[0])

# add objects (class, xmin, ymin, xmax, ymax)
writer.addObject('fire', blobRectangle[0], blobRectangle[1], blobRectangle[0]+blobRectangle[2], blobRectangle[1]+blobRectangle[3])

# write to file
writer.save('XMLs/' + colorList[0][:-4] + '.xml')